<a href="https://colab.research.google.com/github/lucianosilva-github/visaocomputacional/blob/master/Aula%2015/Visa%CC%83o_Computacional_Aula_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **VISÃO COMPUTACIONAL - AULA 15**

**Objetivos da aula:**

*   entender o processo de formação de imagens em câmeras
*   praticar com o gerador de imagens pyRT



**PRINCÍPIOS DE GERAÇÃO DE IMAGENS**

Assim como o Processamento de Imagens e Visão Computacional fazem parte da área de Computação Visual, a Computação Gráfica (Graphics) também pertence a esta área.

<img src="https://camo.githubusercontent.com/6257d85ea432469cd378e8c0a2a7ba1901577465/68747470733a2f2f73746f726167652e676f6f676c65617069732e636f6d2f74656e736f72666c6f772d67726170686963732f6769742f726561646d652f67726170686963732e6a7067"> </img>

Em Computação Gráfica, para se gerar uma imagem a partir de um processo chamado rendering (renderização), necessitamos de alguns elementos importantes: objetos 3D e cenários (geometria), transformações, câmeras, luz e textura.

Um dos elementos centrais neste processo de geração de imagens são as câmeras. Imagens percebidas por uma câmera são projeções de objetos 3D no plano da imagem.

<img src="https://storage.googleapis.com/tensorflow-graphics/notebooks/intrinsics/camera_model.jpg"> </img>

A especificação de uma câmera consiste de parâmetros geométricos (posição, ponto focal) e físicos (abertura, distância focal). 





**ALGORITMO DE RAY TRACING PARA PRODUÇÃO DE IMAGENS**

O Algoritmo de Ray Tracing permite gerar imagens sintéticas a partir de especificações geométricas e fisicas de objetos e câmeras. Este algoritmo permite gerar imagens fotorealísticas e, atualmente, possui implementações em hardware como na placa de vídeo NVidia RTX 2080.

<img src='https://therese3dblog.files.wordpress.com/2014/04/raytracing.gif'> </img>

Para cada pixel da imagem da ser produzida, lançamos um raio da câmera para o pixel, traçamos o acompanhamento deste raio e calculamos a contribuição de cor de cada uma das interseções do raio com os objetos em cena. 

Para entendermos melhor este algoritmo, vamos utilizar o módulo **pvtrace**, disponível gratuitamente para ambientes Python.




In [ ]:
!pip install pyRT

     |████████████████████████████████| 40kB 1.7MB/s 


In [ ]:
from pyrt.math import *
from pyrt.scene import *
from pyrt.geometry import Sphere
from pyrt.material import PhongMaterial
from pyrt.camera import PerspectiveCamera
from pyrt.renderer import SimpleRT

from PIL import Image


# Specify width/height as in example 5
width = 320
height = 240

# now create a camera and a view like in example 5:
camera = PerspectiveCamera(width, height, 60)
camera.setView(Vec3(0,-10,0), Vec3(0,0,0), Vec3(0,0,1))

# Create a scene
scene = Scene()

# Add a sphere to the scene:
scene.add(Sphere(center=Vec3(0.,0.,0.), radius=3., material=PhongMaterial(color=Vec3(1.,0.,0.))))

# Now tell the scene which camera we use
scene.setCamera(camera)

# Create a raytracer using "SimpleRT"
engine = SimpleRT()

# Render the scene:
image = engine.render(scene)

# Save the resulting image using pillow
image.save("render1.png")



# Creating Renderer: Simple Raytracer
# RENDER STATISTICS###############################
TIME FOR RENDERING: 1.7968637943267822s
NUMBER OF PRIMARY RAYS: 76800
NUMBER OF SECONDARY RAYS: 0
NUMBER OF SHADOW RAYS: 0
RAYS/s: 42741.136107522325
##################################################


In [ ]:
from pyrt.math import Vec3
from pyrt.scene import Scene
from pyrt.light import PointLight
from pyrt.geometry import Triangle, Sphere, Vertex
from pyrt.material import PhongMaterial
from pyrt.camera import PerspectiveCamera
from pyrt.renderer import SimpleRT
from PIL import Image

# Specify width/height 
width = 320
height = 240

# now create a camera and a view :
camera = PerspectiveCamera(width, height, 45)
camera.setView(Vec3(0.,-10.,10.), Vec3(0.,0.,0.), Vec3(0.,0.,1.))

# Create a scene
scene = Scene()

# Add a light to the scene
scene.addLight(PointLight(Vec3(0,0,15)))

# create some materials:
floormaterial = PhongMaterial(color=Vec3(0.5,0.5,0.5))
sphere0material = PhongMaterial(color=Vec3(1.,0.,0.))
sphere1material = PhongMaterial(color=Vec3(0.,1.,0.))
sphere2material = PhongMaterial(color=Vec3(0.,0.,1.))
sphere3material = PhongMaterial(color=Vec3(1.,1.,0.))

# Add "floor"

A = Vertex(position=(-5.0, -5.0, 0.0))
B = Vertex(position=( 5.0, -5.0, 0.0))
C = Vertex(position=( 5.0,  5.0, 0.0))
D = Vertex(position=(-5.0,  5.0, 0.0))

scene.add(Triangle(A,B,C, material=floormaterial))
scene.add(Triangle(A,C,D, material=floormaterial))

# Add some spheres

scene.add(Sphere(center=Vec3(-2.5,-2.5,1.75), radius=1.75, material=sphere0material))
scene.add(Sphere(center=Vec3( 2.5,-2.5,1.75), radius=1.75, material=sphere1material))
scene.add(Sphere(center=Vec3( 2.5, 2.5,1.75), radius=1.75, material=sphere2material))
scene.add(Sphere(center=Vec3(-2.5, 2.5,1.75), radius=1.75, material=sphere3material))

# Now tell the scene which camera we use
scene.setCamera(camera)

# Create a raytracer using "SimpleRT"
engine = SimpleRT(shadow=True)

# Render the scene:
image = engine.render(scene)

# Save the resulting image using pillow
image.save("render2.png")


# Creating Renderer: Simple Raytracer
# Shadow Enabled
# RENDER STATISTICS###############################
TIME FOR RENDERING: 10.330995798110962s
NUMBER OF PRIMARY RAYS: 76800
NUMBER OF SECONDARY RAYS: 0
NUMBER OF SHADOW RAYS: 55280
RAYS/s: 12784.82757917209
##################################################


### **EXERCÍCIO**

Especifique uma condfiguração de render para produzir a seguinte imagem:

<img src='https://github.com/martinchristen/pyRT/blob/master/examples/13.png?raw=true'></b>

In [ ]:
#implemente sua solução aqui...

### **EXERCÍCIO**

Especifique uma condfiguração de render para produzir a seguinte imagem:

<img src='https://github.com/martinchristen/pyRT/blob/master/examples/15.png?raw=true'></b>

In [ ]:
#implemente sua solução aqui...